Rodzaje ruchu:\
Swap - losujemy dwa miasta i zamieniamy je ze sobą.\
Insercja - losujemy miasto i punkt w którym je umieszczamy.\
Zamiana kolejności - losujemy dwa punkty i między nimi zamieniamy kolejność.\
\
Sąsiedztwo to dwa przypadki które różnią się między sobą jednym ruchem\
\
Klasyczny algorytm wspinaczki z multistartem: po każdym ruchu liczymy drogę i sprawdzamy czy jest mniejsza czy nie, wybieramy tylko lepsze opcje. Multistart polega na wielokrotnym odpaleniu algorytmu z losowaniem punktu startowego.\
\
Algorytm symulowanego wyżarzania: zmodyfikowany algorytm wspinaczki, jeśli znalezione rozwiązanie jest gorsze to nadal je rozważamy, na podstawie tego jak bardzo jest gorsze oraz od temperatury.\
Temperatura jest wysoka na początku działania algorytmu i spada razem z czasem wykonywania algorytmu, im niższa temperatura tym mniejsze prawdopodobieństwo wybrania gorszego rozwiązania, bo algorytm już trochę działa i zakładamy że jest w lepszym miejscu niż na początku.\
Pozwala on nam wyjść z niektórych ekstremów lokalnych dzięki możliwości przyjęcia gorszego rozwiązania.\
\
Tabu search:\
Sąsiedztwo wygląda tutaj inaczej: przy swapie losujemy jedną liczbę i zamieniamy wszystkie kombinacje z tą liczbą, ze wszystkich kombinacji wybieramy najlepszą.\
Następnie taka kombinacja trafia na listę tabu, czyli takiej samej zamiany nie można wykonać przez określoną kolejną liczbę ruchów.\
\
Funkcja aspiracji to funkcja, która pod pewnymi warunkami pozwala wykonać ruch z listy tabu ale nie trzeba tego implementować.\
\
Odpowiednia długość listy tabu pozwala wyjść z niektórych ekstremów lokalnych, jeśli będzie za krótka to możemy nie wyjść z ekstremum lokalnego, jeśli będzie za długa to możemy wyjść z ekstremum niepotrzebnie.


In [306]:
import numpy as np
import pandas as pd
import math
import random

In [307]:
#wspinaczka z multistartem
distance = np.array([[0,2,4,1,5],
                    [2,0,3,8,7],
                    [4,3,0,5,1],
                    [1,8,5,0,3],
                    [5,7,1,3,0]])

In [308]:
def Odleglosc(distance, trasa):
    suma = 0
    n = len(trasa)
    
    for i, current_city in enumerate(trasa):
        next_city = trasa[(i + 1) % n]  # Use modulo to handle the last element
        suma += distance[current_city][next_city]

    return suma

In [309]:
#swap
def Swap(trasa):
    trasa_temp = trasa.copy()
    #print(trasa)
    swap = np.random.choice(trasa_temp,2,replace=False)
    temp = trasa_temp[swap[0]]
    trasa_temp[swap[0]] = trasa_temp[swap[1]]
    trasa_temp[swap[1]] = temp
    #print(swap)
    #print(trasa)
    return trasa_temp

#insercja
def Insercja(trasa):
    trasa_temp = trasa.copy()
    insercja = np.random.choice(trasa_temp,2,replace=False)
    #print(trasa_temp)
    #print(insercja)
    temp = trasa_temp[insercja[0]]
    trasa_temp = np.delete(trasa_temp,insercja[0])
    trasa_temp = np.insert(trasa_temp,insercja[1],temp)
    #print(trasa_temp)
    return trasa_temp

def Zamiana(trasa):
    trasa_temp = trasa.copy()
    zamiana = np.random.choice(trasa_temp,2,replace=False)
    zamiana.sort()
    #print(trasa_temp)
    #print(zamiana)
    for i in range(math.ceil((zamiana[1]-zamiana[0])/2)):
        temp = trasa_temp[zamiana[1]]
        trasa_temp[zamiana[1]] = trasa_temp[zamiana[0]]
        trasa_temp[zamiana[0]] = temp
        zamiana[1] -= 1
        zamiana[0] += 1
    #print(trasa_temp)
    return trasa_temp

In [310]:
#Zamiana(trasa)

In [311]:
#trasa = np.arange(len(distance))
#np.random.shuffle(trasa)
#Insercja(trasa)

In [312]:
#temp = trasa[0] #temp = 4
#trasa = np.delete(trasa,0)
#trasa = np.insert(trasa,4,3)

In [313]:
#ruch polegający na swapie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_swap(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Swap(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl, trasa_new, odl_new

#ruch polegający na insercji, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_insercja(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Insercja(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new
    return trasa, odl

#ruch polegający na zamianie, sprawdzenie sąsiadów, zwrot mniejszej trasy
def Ruch_zamiana(distance,trasa):
    odl = Odleglosc(distance,trasa)
    trasa_new = Zamiana(trasa)
    odl_new = Odleglosc(distance,trasa_new)
    if odl_new < odl:
        return trasa_new, odl_new  
    return trasa, odl

In [314]:
distance = pd.read_excel("Dane_TSP_48.xlsx")
distance = distance.to_numpy()
distance = np.delete(distance,0,1)
distance

array([[   0, 1495,  381, ...,  488,  753, 1184],
       [1495,    0, 1135, ..., 1727, 1388,  661],
       [ 381, 1135,    0, ...,  640,  591,  810],
       ...,
       [ 488, 1727,  640, ...,    0,  520, 1242],
       [ 753, 1388,  591, ...,  520,    0,  801],
       [1184,  661,  810, ..., 1242,  801,    0]], dtype=int64)

In [315]:
def IHC(distance, ruch, n_iter_ruch, n_iter):  # ruch = {Swap, Insercja, Zamiana}
    min_iter = []
    for i in range(n_iter):
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                if odl_new < odl:
                    odl = odl_new
                    trasa = trasa_new
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = IHC(distance,Swap,1000,100)
wynik.sort(key=lambda x: x[2])
wynik[0:10]



[(14,
  array([26, 42, 18, 36,  5, 29,  6, 35, 43, 30,  8,  7,  0, 21, 40,  1, 41,
          9, 25,  3, 34, 44, 38, 23, 31, 47, 33, 28,  4, 24, 22, 46, 10, 11,
         39, 19, 32, 45, 17, 37, 14, 15,  2, 13, 12, 20, 27, 16]),
  16104),
 (4,
  array([46, 38, 31, 23,  9, 34, 44,  3, 25, 19, 29, 26, 36, 17, 45,  0,  8,
         21, 40, 15, 39, 43,  6, 27,  5, 18, 16, 42, 35, 30, 32, 11, 14, 37,
          7, 22,  2, 33, 13, 12, 20, 24, 47, 41, 28,  1,  4, 10]),
  17178),
 (7,
  array([ 8, 37,  7, 30, 35,  6, 17, 43, 45, 27, 26, 18, 36, 16, 42, 31,  1,
         28, 40,  4, 23, 44, 34, 25,  3,  9, 41, 47, 33, 22, 46, 19, 32,  5,
         29, 12, 24, 38, 20, 11, 14,  0, 15, 21,  2, 10, 13, 39]),
  17383),
 (26,
  array([17, 29,  5, 32,  2, 40, 15,  0,  8, 30, 43,  6, 36, 26, 42, 16, 18,
         27, 35, 39, 22, 13, 44, 34,  9, 23, 31, 38, 20, 24, 33, 12, 11, 10,
         47, 41, 25,  3,  1,  4, 28, 21,  7, 37, 14, 45, 19, 46]),
  17514),
 (34,
  array([37,  7, 15, 21,  2, 22, 10, 46, 20, 19,

In [316]:
def Geometric(T,alpha):
    return alpha*T

def Slow(T,alpha):
    return T/(1+alpha*T)


In [317]:
def SA(distance, ruch, n_iter_ruch, n_iter, T_start, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        for j in range(n_iter_ruch):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,1000,100,100,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(15,
  array([27, 36, 26, 42, 47,  4, 25,  3,  1, 28, 21, 15, 40, 13, 12, 24, 33,
         20, 38, 31, 23, 34, 44,  9, 41, 46, 19, 10,  2, 22, 39, 37, 30,  7,
          0,  8, 11, 14, 32, 45, 17,  6, 43, 35, 16, 18,  5, 29]),
  16559),
 (33,
  array([27, 19, 31, 23, 44,  9, 34,  3, 25,  1, 41,  4, 47, 33, 12, 20, 38,
         24, 22, 39, 45,  8, 21, 15,  0,  7, 37, 43, 17,  6, 18, 16, 42, 26,
         30, 40, 28,  2, 13, 10, 11, 46, 14, 32, 35, 29, 36,  5]),
  16796),
 (17,
  array([30, 43,  6, 18, 16, 29, 36, 27, 32, 39, 14, 10, 11,  5, 42, 26, 35,
         45,  8,  0,  7, 21, 15,  2, 40, 33,  1, 41, 25,  9, 28, 13, 24, 12,
         22, 46, 31, 47, 23, 44, 34,  3,  4, 38, 20, 19, 17, 37]),
  16902),
 (14,
  array([47, 38, 33, 21,  8, 39, 45, 32, 11, 14, 37, 43, 30, 42, 16, 26, 35,
         17,  6, 27, 29, 18, 36,  5, 19, 20, 31, 44, 34,  9, 23,  3, 25,  4,
         13, 22, 10, 12, 46, 24, 28,  1, 41,  2,  7,  0, 15, 40]),
  17044),
 (25,
  array([30, 43, 17, 19, 41,  9,  3, 25,  1, 2

In [318]:
# wersja SA gdzie warunkiem stopu jest temperatura a nie liczba iteracji 
# (chyba lepiej tak bo upewniamy sie ze temperatura jest niska a podajac liczbe iteracji mozemy skonczyc na wysokiej temperaturze)
# mozna tez dodac warunek stopu ze po np 100 iteracjach bez poprawy rozwiazania to koniec

def SA(distance, ruch, n_iter, T_start, T_stop, cooling, alpha):  # ruch = {Swap, Insercja, Zamiana}, cooling = {Geometric, Slow}, alpha = cooling param 
    min_iter = []
    for i in range(n_iter):
        T = T_start
        trasa = np.arange(len(distance))
        np.random.shuffle(trasa)
        odl = Odleglosc(distance,trasa)
        while(T > T_stop):
                trasa_new = ruch(trasa)
                odl_new = Odleglosc(distance,trasa_new)
                diff_odl = odl_new - odl
                if diff_odl < 0:
                    odl = odl_new
                    trasa = trasa_new
                elif random.random() < math.exp(-diff_odl/T):
                    odl = odl_new
                    trasa = trasa_new
                T = cooling(T,alpha)
        min_iter.append((i,trasa,odl))
    return min_iter
wynik = SA(distance,Swap,100,100,0.01,Slow,0.1)
wynik.sort(key=lambda x: x[2])
wynik[0:10]

[(54,
  array([47, 38, 28, 33, 21, 15, 40, 41,  1,  3, 25, 44, 34,  9, 23, 31, 46,
         14,  8,  0,  7, 37, 30, 43,  6, 45, 32, 11, 17, 35, 36, 18, 29, 16,
         42, 26,  5, 27, 19, 13,  4, 24, 12, 39,  2, 22, 10, 20]),
  16159),
 (82,
  array([47, 41,  9, 34, 44,  3, 25,  1, 28,  4, 33,  2, 40, 13, 24, 31, 38,
         29, 26, 18, 16, 19, 20, 46, 32, 45,  8, 14,  0,  7, 39, 37, 30, 43,
         17,  6, 27, 36,  5, 42, 35, 15, 21, 22, 10, 11, 12, 23]),
  16618),
 (20,
  array([41, 25,  3,  1, 15, 21, 39, 14, 45, 32,  5, 16,  6, 37, 30, 36, 18,
         26, 42, 29, 27, 19, 31, 38, 47, 24, 13, 10, 11, 17, 35, 43,  8,  7,
          0,  2, 22, 12, 20, 46, 33, 40, 28,  9, 34, 44, 23,  4]),
  16648),
 (27,
  array([10, 19, 46, 20, 12, 24, 22,  7, 15, 21, 30,  5, 18, 36, 35, 17, 37,
          8,  0,  2, 40,  1, 41, 38, 31, 11, 32, 45, 43, 42, 16, 26, 29, 27,
          6, 14, 39, 33, 28, 23,  9, 25,  3, 34, 44, 47,  4, 13]),
  16699),
 (33,
  array([10, 45, 17, 35, 32, 19, 46,  4,  9, 4

In [319]:
# funkcje pomocnicze do sprawdzenia ile iteracji odpowiada parametrowi T_stop
def T_stopToIter(T_start, T_stop, cooling, alpha):
    T = T_start
    i = 0
    while(T > T_stop):
        T = cooling(T,alpha)
        i += 1
    return i
def IterToT_stop(T_start, n_iter, cooling, alpha):
    T = T_start
    for i in range(n_iter):
        T = cooling(T,alpha)
    return T
print(T_stopToIter(100,0.01,Slow,0.1))
IterToT_stop(100,100,Slow,0.1)

1000


0.09990009990009993

algorytm genetyczny

In [320]:
def ox(r1, r2):
    n = len(r1)
    a = random.randint(0,n-1)
    b = random.randint(0,n-1)
    if a > b:
        a,b = b,a
    p1 = r1.copy(); p2 = r2.copy()
    p1[a:b] = r2[a:b]; p2[a:b] = r1[a:b]
    return p1, p2

In [341]:
r1 = [1,2,3,4,5,6,7,8,9]
r2 = [4,5,2,1,8,7,6,9,3]
n = len(r1)
a = 3
b = 7
section = np.arange(a)
section = np.append(section,np.arange(b,n))
if a > b:
    a,b = b,a
p1 = r1.copy()
p2 = r2.copy()
p1[0:a], p1[b:n] = np.zeros(a).astype(int), np.zeros(n-b).astype(int)
p2[0:a], p2[b:n] = np.zeros(a).astype(int), np.zeros(n-b).astype(int)
print(p1,p2)

j = 0
for i in section:
    while True:
        if r2[i] not in p1:
            p2[i] = r1[j]
            j+=1
            break
        else:
            j+=1
print(p1,p2)

[0, 0, 0, 4, 5, 6, 7, 0, 0] [0, 0, 0, 1, 8, 7, 6, 0, 0]


In [322]:
# #algorytm ze swapem
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_swap(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))


In [323]:
# #algorytm z insercją
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 5

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_insercja(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [324]:
# #algorytm z zamianą
# trasa = np.arange(len(distance))
# np.random.shuffle(trasa)
# licza_iteracji = 1000

# print("Start")
# print(trasa)
# print(Odleglosc(distance,trasa))
# for i in range(licza_iteracji):
#     print("\n{} iteracja".format(i+1))
#     trasa = Ruch_zamiana(distance,trasa)
#     print(trasa)
#     print(Odleglosc(distance,trasa))

In [325]:
#NN
min_odl = []
for n in range(len(distance)):
    potencjalne = np.arange(len(distance))
    trasa = np.zeros(len(distance),dtype=int)
    trasa[0] = potencjalne[n]
    potencjalne = np.delete(potencjalne,int(trasa[0]))

    for cur in range(1,len(trasa)):
        odleglosci = []
        for i in range(len(potencjalne)):
            trasa[cur] = potencjalne[i]
            odleglosci.append((potencjalne[i],Odleglosc(distance,trasa[0:cur+1])))
        odleglosci.sort(key=lambda x: x[1])
        trasa[cur] = odleglosci[0][0]
        potencjalne = np.delete(potencjalne,potencjalne==trasa[cur])
    min_odl.append((Odleglosc(distance,trasa),trasa))
    
min_odl.sort(key=lambda x:x[0])
min_odl[:10]

KeyboardInterrupt: 